In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# # Importing the chembl_database
! pip install chembl_webresource_client

# !pip install requests-cache==1.1.1
# !pip install chembl-webresource-client==0.10.8


In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [ ]:
# PART 1

# Target search for hcm using tnni3
target = new_client.target
target_query = target.search('tnni3')
targets = pd.DataFrame.from_dict(target_query)
targets

In [ ]:
# Retrieve data for the single protein for humans
selected_target = targets.target_chembl_id[2]
selected_target

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)
df.head()

In [ ]:
# Filtering the ones with unique value of IC50
df.standard_type.unique()

In [ ]:
df.to_csv('bioactivity_data.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dropping any values without the standard type
# Dropping rows with null values for standard type

df2 = df[df.standard_value.notna()]
df2

In [ ]:
# PART 2

# DATA PRE-PROCESSING

# Compounds are segregated based on a certain criteria of IC50 value

# Segregation is happening based on standard value

# Compounds having values of less than 1000 nM will be considered to be active
# those greater than 10,000 nM will be considered to be inactive.
# values in between 1,000 and 10,000 nM will be referred to as intermediate.

bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

In [ ]:
#Iterate the "molecule_chembl_id" to a list
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

In [ ]:
# Iterate "canonical_smiles" to a list
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

In [ ]:
# Iterate standard_value to a list
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

In [ ]:
# Combine the 4 lists into a dataframe
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])
df3

In [ ]:
# storing df as a csv file
df3.to_csv('bioactivity_data_preprocessed.csv', index=False)

In [ ]:
!wget https://your_download_link.com/padel.zip
!unzip padel.zip -d PaDEL-Descriptor


In [ ]:
! ls -l

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
df = pd.read_csv('bioactivity_data_preprocessed.csv')

In [ ]:
!pip install rdkit-pypi


In [ ]:
# PART 3

# Calculating Lipinski descripters
# Describes the drug-likeness of a compound. Based on ADME (Absorptive, Distribution, Metabolism, and Excretion).
# The rule stays fixed where properties such as H-bond and molecular weight are to be lesser than a certain value only and such.

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

print("RDKit is working!")


In [ ]:
# To calculate the Lipinski descripters
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors

In [ ]:
# Provides exact atomic details
df_lipinski = lipinski(df.canonical_smiles)

# Further details as per the Lipinski parameters
df_lipinski

In [ ]:
# The lipinski values and the original dataframe are combined to an all in one
df_combined = pd.concat([df,df_lipinski], axis=1)
df_combined.to_csv('df_combined.csv')
df_combined

In [ ]:
# Convert ic50 to pic50
# pic50 is bsaically -log10 of ic50


def pIC50(input):
    pIC50 = []

    for i in input['standard_value']:
        molar = i * (10**-9)  # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50

    # ✅ Fix: Use `axis=1` explicitly
    x = input.drop('standard_value', axis=1)

    return x


In [ ]:
# pic50 is performed on standard value in df_combined
df_final = pIC50(df_combined)
df_final

In [ ]:
# PART 4

# Chemical space analysis w lipinski descriptors
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

In [ ]:
# Frequency plot of the combined data
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='bioactivity_class', data=df_combined, edgecolor='black')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.pdf')

In [ ]:
# Box plot for all the three variants wrt to their type and their respective standard values
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'standard_value', data = df_combined)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('IC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')

In [ ]:
# Box plots for the other 4 Lipinski descripters

# M W

plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'MW', data = df_combined)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

In [ ]:
# Log P
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'LogP', data = df_combined)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')

In [ ]:
# NumHDonors
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHDonors', data = df_combined)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.pdf')

In [ ]:
# NumHAcceptors
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHAcceptors', data = df_combined)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.pdf')

In [ ]:
! zip -r results.zip . -i *.csv *.pdf

In [ ]:
# PART 5
# lipinski descriptors provide very simple molecular descriptor overview of the drug like properties
# padel will be used to calculate molecular descriptors

!wget /content/drive/MyDrive/padel.zip

In [ ]:
! unzip padel.zip

In [ ]:
# The same file as that of df_combined
df4=pd.read_csv('df_combined.csv')

In [ ]:
# canonical_smiles and molecule_chembl_id is filtered and placed into a new variable
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df4[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
# It contains the smiles notation and the particular molecule
# Smiles notation represents the chemical structure

! cat molecule.smi | head -5

In [ ]:
!ls

In [ ]:
# Cleans the data of any impurities like salts,acids and such.
# It then calculates the finger print values of the type pubchem finger prints

! cat padel.sh

In [ ]:
! bash padel.sh

In [ ]:
import os
print(os.listdir())  # Lists all files in the current directory


In [ ]:
# Preparing data matrices

# This matrix consists of the data descriptors (pubchem fingerprints) for every molecule
# Pubchem fingerprints are molecular features.

dataset3 = pd.read_csv('descriptors_output.csv')
dataset3

In [ ]:
dataset3.to_csv('final_matrix_ic50_only.csv', index=False)

In [ ]:
dataset3 = dataset3.drop(columns=['Name'])

In [ ]:
# PART 6

# Model building
# X - pubchem finger prints
# Y - pic50 values

X = dataset3
Y = df_final.pIC50
X.shape
Y.shape

In [ ]:
# lipinski descriptors majorly talk about the global aspects of a molecule like solubility and h bond related stuff
# whereas pubchem fingerprints prev created talk more of the local aspects such as within the molecule prop stuff

In [ ]:
# Remove low variance features
# Reduces the number of columns

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import sklearn
import math

selection = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = selection.fit_transform(X)

In [ ]:
X.shape

In [ ]:
# Data split 8:2 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 1)
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

In [ ]:
# Regression model using Random Forest
import numpy as np
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
Y_pred

In [ ]:
mse = sklearn.metrics.mean_squared_error(Y_test,Y_pred)
rmse = math.sqrt(mse)
print('Accuracy for Random Forest',max(0,rmse))

In [ ]:
# Scatter plot of Experimental vs Predicted pic50 values

sns.set(color_codes=True)
sns.set_style("ticks")

ax = sns.regplot(Y_test, Y_pred, scatter_kws={'alpha':0.4})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(5, 5)
plt.legend(["Experimental pIC50","Predicted pIC50"])
plt.show